In [18]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute, roll_time_series
from tsfresh.feature_extraction import EfficientFCParameters
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [19]:
#Function to load and process data with op settings
def load_and_process_data(file_path):
    col_1 = ['unit', 'time', 'op1', 'op2', 'op3']  
    col_2 = [f'sr{i+1}' for i in range(21)] 
    columns = col_1 + col_2
    df = pd.read_csv(file_path, delim_whitespace=True, header=None, names=columns)
    df['max_time'] = df.groupby('unit')['time'].transform('max')
    df['remaining_time'] = df['max_time'] - df['time']
    df['label'] = df['remaining_time'].clip(upper=130)
    return df

train_df_with_op = load_and_process_data(r"C:\Users\65962\Desktop\JUPYTER\CMAPSSData\train_FD003.txt")
train_df_with_op

C:\Users\65962\AppData\Local\Temp\ipykernel_13436\345789304.py:6: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(file_path, delim_whitespace=True, header=None, names=columns)


,unit,time,op1,op2,op3,sr1,sr2,sr3,sr4,sr5,...,sr15,sr16,sr17,sr18,sr19,sr20,sr21,max_time,remaining_time,label
0,1,1,-0.0005,0.0004,100.0,518.67,642.36,1583.23,1396.84,14.62,...,8.4246,0.03,391,2388,100.0,39.11,23.3537,259,258,130
1,1,2,0.0008,-0.0003,100.0,518.67,642.50,1584.69,1396.89,14.62,...,8.4403,0.03,392,2388,100.0,38.99,23.4491,259,257,130
2,1,3,-0.0014,-0.0002,100.0,518.67,642.18,1582.35,1405.61,14.62,...,8.3901,0.03,391,2388,100.0,38.85,23.3669,259,256,130
3,1,4,-0.0020,0.0001,100.0,518.67,642.92,1585.61,1392.27,14.62,...,8.3878,0.03,392,2388,100.0,38.96,23.2951,259,255,130
4,1,5,0.0016,0.0000,100.0,518.67,641.68,1588.63,1397.65,14.62,...,8.3869,0.03,392,2388,100.0,39.14,23.4583,259,254,130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24715,100,148,-0.0016,-0.0003,100.0,518.67,643.78,1596.01,1424.11,14.62,...,8.5036,0.03,394,2388,100.0,38.44,22.9631,152,4,4
24716,100,149,0.0034,-0.0003,100.0,518.67,643.29,1596.38,1429.14,14.62,...,8.5174,0.03,395,2388,100.0,38.50,22.9746,152,3,3
24717,100,150,-0.0016,0.0004,100.0,518.67,643.84,1604.53,1431.41,14.62,...,8.5223,0.03,396,2388,100.0,38.39,23.0682,152,2,2
24718,100,151,-0.0023,0.0004,100.0,518.67,643.94,1597.56,1426.57,14.62,...,8.5148,0.03,395,2388,100.0,38.31,23.0753,152,1,1


In [20]:
#summary stats
def display_summary_stats(df):
    print(df.describe())

print(display_summary_stats(train_df_with_op))

               unit          time           op1           op2      op3  \
count  24720.000000  24720.000000  24720.000000  24720.000000  24720.0   
mean      48.631877    139.077063     -0.000024      0.000005    100.0   
std       29.348985     98.846675      0.002194      0.000294      0.0   
min        1.000000      1.000000     -0.008600     -0.000600    100.0   
25%       23.000000     62.000000     -0.001500     -0.000200    100.0   
50%       47.000000    124.000000     -0.000000     -0.000000    100.0   
75%       74.000000    191.000000      0.001500      0.000300    100.0   
max      100.000000    525.000000      0.008600      0.000700    100.0   

            sr1           sr2           sr3           sr4           sr5  ...  \
count  24720.00  24720.000000  24720.000000  24720.000000  2.472000e+04  ...   
mean     518.67    642.457858   1588.079175   1404.471212  1.462000e+01  ...   
std        0.00      0.523031      6.810418      9.773178  3.552786e-15  ...   
min      518.

In [21]:
# Feature Extraction with rolling windows
def feature_extraction(df, window_size=30):
    rolled_train = roll_time_series(df, column_id='unit', column_sort='time', max_timeshift=window_size - 1, min_timeshift=window_size - 1, rolling_direction=1)
    selected_cols = ['unit', 'label', 'op1', 'op2', 'op3'] + [f'sr{i+1}' for i in range(21)]
    selected_train = rolled_train[selected_cols]

    extraction_settings = EfficientFCParameters()
    X_train_features = extract_features(selected_train,
                                        column_id='unit',
                                        impute_function=impute,
                                        default_fc_parameters=extraction_settings)

    #drop cols with NaNs
    X_train_features = X_train_features.dropna(axis=1, how='all')  
    return X_train_features, rolled_train.groupby('unit')['label'].last()

X_trainwithop, y_trainwithop=feature_extraction(train_df_with_op, window_size=30)
X_trainwithop, y_trainwithop

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 40/40 [05:03<00:00,  7.59s/it]


(     op2__variance_larger_than_standard_deviation  op2__has_duplicate_max  \
 1                                             0.0                     1.0   
 2                                             0.0                     1.0   
 3                                             0.0                     1.0   
 4                                             0.0                     1.0   
 5                                             0.0                     1.0   
 ..                                            ...                     ...   
 96                                            0.0                     1.0   
 97                                            0.0                     1.0   
 98                                            0.0                     1.0   
 99                                            0.0                     1.0   
 100                                           0.0                     1.0   
 
      op2__has_duplicate_min  op2__has_duplicate  op2__sum_val

In [40]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression
# Feature Selection
def select_top_features(X, y, k=10):
    selector = SelectKBest(score_func=mutual_info_regression, k=k)
    X_selected = selector.fit_transform(X, y)
    selected_feature_names = X.columns[selector.get_support()]  # Names of selected features
    print(f"Selected Features: {selected_feature_names}")
    return X_selected, selected_feature_names

X_selected_with_op, selected_feature_nameswithop=select_top_features(X_trainwithop, y_trainwithop, k=10)
X_selected_with_op, selected_feature_nameswithop

Selected Features: Index(['op3__index_mass_quantile__q_0.2', 'op3__index_mass_quantile__q_0.4',
       'op3__energy_ratio_by_chunks__num_segments_10__segment_focus_5',
       'sr1__partial_autocorrelation__lag_6', 'sr3__symmetry_looking__r_0.9',
       'sr5__change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.6',
       'sr8__large_standard_deviation__r_0.5',
       'sr10__agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"var"',
       'sr15__symmetry_looking__r_0.55',
       'sr19__large_standard_deviation__r_0.9500000000000001'],
      dtype='object')


(array([[2.00000000e-01, 4.00000000e-01, 1.00000000e-01, 0.00000000e+00,
         1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         1.00000000e+00, 0.00000000e+00],
        [2.00000000e-01, 4.00000000e-01, 1.00000000e-01, 0.00000000e+00,
         1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.19186087e-10,
         1.00000000e+00, 0.00000000e+00],
        [2.00000000e-01, 4.00000000e-01, 1.00000000e-01, 0.00000000e+00,
         1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         1.00000000e+00, 0.00000000e+00],
        [2.00000000e-01, 4.00000000e-01, 1.00000000e-01, 0.00000000e+00,
         1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         1.00000000e+00, 0.00000000e+00],
        [2.00000000e-01, 4.00000000e-01, 1.00000000e-01, 0.00000000e+00,
         1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         1.00000000e+00, 0.00000000e+00],
        [2.00000000e-01, 4.00000000e-01, 1.00000000e-01, 0.0

In [41]:
# Data scaling and train-validation split
def scale_split(X, y, test_size=0.2):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return train_test_split(X_scaled, y, test_size=test_size, random_state=42)

X_train_splitwithop, X_val_splitwithop, y_train_splitwithop, y_val_splitwithop = scale_split(X_selected_with_op, y_trainwithop)

In [48]:
# Model training
from sklearn.metrics import mean_squared_error, r2_score
def train_xgb_model(X_train, y_train, X_val, y_val):
    xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
    xgb_model.fit(X_train, y_train)

    y_pred_val = xgb_model.predict(X_val)
    mse_val = mean_squared_error(y_val, y_pred_val)
    r2_val = r2_score(y_val, y_pred_val)
    print(mse_val,r2_val)
    return xgb_model, y_pred_val

xgb_modelwithop, y_pred_withop = train_xgb_model(X_train_splitwithop, y_train_splitwithop, X_val_splitwithop, y_val_splitwithop)
print(xgb_modelwithop)
print(y_pred_withop)


0.0 1.0
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [28]:
#Function to load and process data without op settings
def load_and_process_withoutop(file_path):
    col_1 = ['unit', 'time', 'op1', 'op2', 'op3']  
    col_2 = [f'sr{i+1}' for i in range(21)] 
    columns = col_1 + col_2
    df = pd.read_csv(file_path, delim_whitespace=True, header=None, names=columns)
    df = df.drop(columns=['op1', 'op2', 'op3'])
    # Calculating remaining useful life (RUL)
    df['max_time'] = df.groupby('unit')['time'].transform('max')
    df['remaining_time'] = df['max_time'] - df['time']
    df['label'] = df['remaining_time'].clip(upper=130)
    return df

train_df_without_op = load_and_process_withoutop(r"C:\Users\65962\Desktop\JUPYTER\CMAPSSData\train_FD003.txt")
train_df_without_op

C:\Users\65962\AppData\Local\Temp\ipykernel_13436\1443521108.py:6: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(file_path, delim_whitespace=True, header=None, names=columns)


,unit,time,sr1,sr2,sr3,sr4,sr5,sr6,sr7,sr8,...,sr15,sr16,sr17,sr18,sr19,sr20,sr21,max_time,remaining_time,label
0,1,1,518.67,642.36,1583.23,1396.84,14.62,21.61,553.97,2387.96,...,8.4246,0.03,391,2388,100.0,39.11,23.3537,259,258,130
1,1,2,518.67,642.50,1584.69,1396.89,14.62,21.61,554.55,2388.00,...,8.4403,0.03,392,2388,100.0,38.99,23.4491,259,257,130
2,1,3,518.67,642.18,1582.35,1405.61,14.62,21.61,554.43,2388.03,...,8.3901,0.03,391,2388,100.0,38.85,23.3669,259,256,130
3,1,4,518.67,642.92,1585.61,1392.27,14.62,21.61,555.21,2388.00,...,8.3878,0.03,392,2388,100.0,38.96,23.2951,259,255,130
4,1,5,518.67,641.68,1588.63,1397.65,14.62,21.61,554.74,2388.04,...,8.3869,0.03,392,2388,100.0,39.14,23.4583,259,254,130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24715,100,148,518.67,643.78,1596.01,1424.11,14.62,21.61,551.86,2388.25,...,8.5036,0.03,394,2388,100.0,38.44,22.9631,152,4,4
24716,100,149,518.67,643.29,1596.38,1429.14,14.62,21.61,551.86,2388.23,...,8.5174,0.03,395,2388,100.0,38.50,22.9746,152,3,3
24717,100,150,518.67,643.84,1604.53,1431.41,14.62,21.61,551.30,2388.25,...,8.5223,0.03,396,2388,100.0,38.39,23.0682,152,2,2
24718,100,151,518.67,643.94,1597.56,1426.57,14.62,21.61,550.69,2388.26,...,8.5148,0.03,395,2388,100.0,38.31,23.0753,152,1,1


In [29]:
#summary stats
print(display_summary_stats(train_df_without_op))

               unit          time       sr1           sr2           sr3  \
count  24720.000000  24720.000000  24720.00  24720.000000  24720.000000   
mean      48.631877    139.077063    518.67    642.457858   1588.079175   
std       29.348985     98.846675      0.00      0.523031      6.810418   
min        1.000000      1.000000    518.67    640.840000   1564.300000   
25%       23.000000     62.000000    518.67    642.080000   1583.280000   
50%       47.000000    124.000000    518.67    642.400000   1587.520000   
75%       74.000000    191.000000    518.67    642.790000   1592.412500   
max      100.000000    525.000000    518.67    645.110000   1615.390000   

                sr4           sr5           sr6           sr7           sr8  \
count  24720.000000  2.472000e+04  24720.000000  24720.000000  24720.000000   
mean    1404.471212  1.462000e+01     21.595841    555.143808   2388.071555   
std        9.773178  3.552786e-15      0.018116      3.437343      0.158285   
min     

In [32]:
# Feature Extraction with rolling windows
def feature_extraction_withoutop(df, window_size=30):
    rolled_train = roll_time_series(df,column_id='unit',column_sort='time',max_timeshift=window_size - 1,min_timeshift=window_size - 1,rolling_direction=1)
    selected_cols = ['unit', 'label'] + [f'sr{i+1}' for i in range(21)]
    selected_train = rolled_train[selected_cols]

    extraction_settings = EfficientFCParameters()
    X_train_features = extract_features(selected_train,
                                        column_id='unit',
                                        impute_function=impute,
                                        default_fc_parameters=extraction_settings)

    #drop cols with NaNs
    X_train_features = X_train_features.dropna(axis=1, how='all')  
    return X_train_features, rolled_train.groupby('unit')['label'].last()

X_trainwithoutop, y_trainwithoutop=feature_extraction_withoutop(train_df_without_op, window_size=30)
X_trainwithoutop, y_trainwithoutop

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 40/40 [04:25<00:00,  6.64s/it]


(     label__variance_larger_than_standard_deviation  label__has_duplicate_max  \
 1                                               1.0                       1.0   
 2                                               1.0                       1.0   
 3                                               1.0                       1.0   
 4                                               1.0                       1.0   
 5                                               1.0                       1.0   
 ..                                              ...                       ...   
 96                                              1.0                       1.0   
 97                                              1.0                       1.0   
 98                                              1.0                       1.0   
 99                                              1.0                       1.0   
 100                                             1.0                       1.0   
 
      label__h

In [44]:
# Feature Selection
X_selected_without_op, selected_feature_nameswithoutop=select_top_features(X_trainwithoutop, y_trainwithoutop, k=10)
X_selected_without_op, selected_feature_nameswithoutop

Selected Features: Index(['sr1__partial_autocorrelation__lag_0', 'sr1__ratio_beyond_r_sigma__r_3',
       'sr4__large_standard_deviation__r_0.4', 'sr9__benford_correlation',
       'sr12__number_crossing_m__m_-1',
       'sr14__large_standard_deviation__r_0.25',
       'sr16__large_standard_deviation__r_0.15000000000000002',
       'sr16__ar_coefficient__coeff_2__k_10', 'sr18__fourier_entropy__bins_3',
       'sr19__index_mass_quantile__q_0.8'],
      dtype='object')


(array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         -2.97356263e-01,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  8.91972250e-04,  0.00000000e+00,
          8.00000000e-01],
        [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         -2.97356263e-01,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  8.91972250e-04,  0.00000000e+00,
          8.00000000e-01],
        [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         -2.97356263e-01,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00, -3.29818726e-02,  0.00000000e+00,
          8.00000000e-01],
        [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         -2.97356263e-01,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  8.91972250e-04,  0.00000000e+00,
          8.00000000e-01],
        [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         -2.97356263e-01,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  8.91972250e-04,  0.000000

In [45]:
# Data scaling and train-validation split
X_train_splitnoop, X_val_splitnoop, y_train_splitnoop, y_val_splitnoop = scale_split(X_selected_without_op, y_trainwithoutop)

In [49]:
# Model training
xgb_modelwithoutop, y_pred_withoutop = train_xgb_model(X_train_splitnoop, y_train_splitnoop, X_val_splitnoop, y_val_splitnoop)
print(xgb_modelwithoutop)
print(y_pred_withoutop)

0.0 1.0
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
